# Sysctl Analysis - Fresh Start

Analyzing sysctl data from `sysctl_analysis.csv`

## Load and Explore Data

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Set font to Helvetica
plt.rcParams['font.family'] = 'Helvetica'

# Load the data
df = pd.read_csv('sysctl_analysis.csv')
print(f"Dataset shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")

Dataset shape: (145, 12)

Columns: ['sysctl', 'End Year', 'Start Year', 'Years', 'End Commit', 'Start Commit', 'Domain', '1:BUG, 0: Fixed', 'years', 'bugFix', 'bugAdd', 'PotentialBUG']


In [2]:
# Filter: Start Year is not empty AND End Year is not empty AND Domain equals 1
domain_change = df[(df['Start Year'].notna()) & (df['End Year'].notna()) & (df['Domain'] == 1)]

print(f"Items with domain change (Domain=1, Start Year and End Year not empty): {len(domain_change)}")
print(f"\nThese items:")
print(domain_change[['sysctl', 'Start Year', 'End Year', 'Domain']].to_string())

Items with domain change (Domain=1, Start Year and End Year not empty): 29

These items:
                       sysctl  Start Year  End Year  Domain
116              rmem_default      2005.0    2024.0     1.0
117                  rmem_max      2005.0    2024.0     1.0
118              wmem_default      2005.0    2024.0     1.0
119                  wmem_max      2005.0    2024.0     1.0
120            tcp_rto_min_us      2005.0    2024.0     1.0
121                optmem_max      2005.0    2023.0     1.0
122               min_adv_mss      2005.0    2022.0     1.0
123               mtu_expires      2005.0    2022.0     1.0
124         tcp_adv_win_scale      2005.0    2017.0     1.0
125               tcp_app_win      2005.0    2017.0     1.0
126   tcp_challenge_ack_limit      2005.0    2017.0     1.0
127                  tcp_frto      2005.0    2017.0     1.0
128                  tcp_rmem      2005.0    2017.0     1.0
129  tcp_thin_linear_timeouts      2005.0    2017.0     1.0
130       t

In [3]:
# Filter: Start Year is not empty AND End Year is not empty AND Domain is empty
const_to_sysctl = df[(df['Start Year'].notna()) & (df['End Year'].notna()) & (df['Domain'].isna())]

print(f"Items changed from constant to sysctl (Domain is empty, Start Year and End Year not empty): {len(const_to_sysctl)}")
print(f"\nThese items:")
print(const_to_sysctl[['sysctl', 'Start Year', 'End Year', 'Domain']].to_string())

Items changed from constant to sysctl (Domain is empty, Start Year and End Year not empty): 20

These items:
                    sysctl  Start Year  End Year  Domain
0          tcp_min_snd_mss      2006.0    2019.0     NaN
1               dev_weight      2005.0    2016.0     NaN
2       dev_weight_rx_bias      2005.0    2016.0     NaN
3       dev_weight_tx_bias      2005.0    2016.0     NaN
4            max_skb_frags      2005.0    2016.0     NaN
5      tcp_mtu_probe_floor      2008.0    2019.0     NaN
6   watermark_scale_factor      2005.0    2016.0     NaN
7      netdev_budget_usecs      2008.0    2017.0     NaN
8       tcp_max_reordering      2005.0    2014.0     NaN
9    sched_rr_timeslice_ms      2005.0    2013.0     NaN
10         max_pages_limit      2018.0    2024.0     NaN
11       extfrag_threshold      2005.0    2010.0     NaN
12     tcp_pingpong_thresh      2019.0    2023.0     NaN
13  dirty_background_bytes      2005.0    2009.0     NaN
14             dirty_bytes      2005

In [4]:
import json

with open('procfs_all.json', 'r') as f:
    data = json.load(f)

# Count all items (last level entries)
total_count = 0
for key, items in data.items():
    total_count += len(items)
    
print(f"Total number of last-level items in procfs_all.json: {total_count}")

Total number of last-level items in procfs_all.json: 702


In [5]:
# Count rows where PotentialBug is 1
potential_bug_count = df[df['PotentialBUG'] == 1]

print(f"Total rows with PotentialBUG = 1: {len(potential_bug_count)}")
print(f"\nBreakdown:")
print(f"  - With actual bug (bugAdd and bugFix not empty): {len(potential_bug_count[(potential_bug_count['bugAdd'].notna()) & (potential_bug_count['bugFix'].notna())])}")
print(f"  - Without actual bug (missing bugAdd or bugFix): {len(potential_bug_count[(potential_bug_count['bugAdd'].isna()) | (potential_bug_count['bugFix'].isna())])}")
print(f"\nThese items:")
print(potential_bug_count[['sysctl', 'Start Year', 'End Year', 'PotentialBUG', 'bugAdd', 'bugFix']].to_string())

Total rows with PotentialBUG = 1: 43

Breakdown:
  - With actual bug (bugAdd and bugFix not empty): 0
  - Without actual bug (missing bugAdd or bugFix): 43

These items:
                                     sysctl  Start Year  End Year  PotentialBUG  bugAdd  bugFix
6                    watermark_scale_factor      2005.0    2016.0             1     NaN     NaN
7                       netdev_budget_usecs      2008.0    2017.0             1     NaN     NaN
9                     sched_rr_timeslice_ms      2005.0    2013.0             1     NaN     NaN
10                          max_pages_limit      2018.0    2024.0             1     NaN     NaN
11                        extfrag_threshold      2005.0    2010.0             1     NaN     NaN
13                   dirty_background_bytes      2005.0    2009.0             1     NaN     NaN
14                              dirty_bytes      2005.0    2009.0             1     NaN     NaN
16                   watermark_boost_factor      2016.0    201